#### Генерируем часть датасета для обучения BERT (для кода, который в репозитории preposition-generation)

Парсим названия блюд с сайта, собираем наборы  {действие + блюдо}, при помощи правил генерируем связанное предложение 

In [ ]:
from bs4 import BeautifulSoup
from lxml import html
import requests
import pandas as pd 
import re

In [ ]:
# 47 = основные блюда 
# 42 = выпечка и сладкое 
# 45 = закуски 
# 76 = итальянская 
# 91 = русская
# 69 = грузинская 
# 56 = американская
# 98 = французская 
# 72 = европейская
# 97 = украинская
# 103 = японская
# 57 = английская 
# 83 = мексиканская 
# 75 = испанская 
# 96= узбекская 

In [3]:
import time
# data = pd.Series()
data_list = []

# category = [76,91,69,42,47,45]
# category = [56,98,72,97,103,57,83,75,96]
category = [77]
for j in category:
    for i in range(10):
        url =f'https://hi-chef.ru/api/recipes/?category={j}&page={i}'
        print(i)
        time.sleep(1)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        res = [i.split('"title":')[1].strip('"') for i in re.findall('"id":[0-9]*,"title":"[А-Яа-я -]*"',soup.text)]

        for i in res:
            print(i)
            data_list.append(i)


data_42 = pd.Series(data_list)

0
1
Мясо в крахмале по-китайски
Суп со шпинатом и соусом
Креветки в кисло-сладком соусе
Битые огурцы
Гречка в азиатском стиле
Гречневая лапша с курицей терияки
Рис с курицей и ананасом
Салат из битых огурцов
Стеклянная лапша с креветками
Китайские квадратики с фаршем
Жареный рис по-китайски
Куриная грудка в кисло-сладком соусе
Фунчоза с овощами и морепродуктами
2
Говядина по-сычуаньски
Китайское печенье с предсказаниями
Кисло-сладкие свиные ребра по-китайски
Свинина в кисло-сладком соусе с ананасом
3
4
5
6
7
8
9


In [5]:
data_42.drop_duplicates()
data_42

0                  Мясо в крахмале по-китайски
1                     Суп со шпинатом и соусом
2               Креветки в кисло-сладком соусе
3                                 Битые огурцы
4                     Гречка в азиатском стиле
5            Гречневая лапша с курицей терияки
6                     Рис с курицей и ананасом
7                       Салат из битых огурцов
8                Стеклянная лапша с креветками
9                Китайские квадратики с фаршем
10                     Жареный рис по-китайски
11        Куриная грудка в кисло-сладком соусе
12          Фунчоза с овощами и морепродуктами
13                      Говядина по-сычуаньски
14          Китайское печенье с предсказаниями
15      Кисло-сладкие свиные ребра по-китайски
16    Свинина в кисло-сладком соусе с ананасом
dtype: object

In [7]:
data_42.to_excel('data_food_names.xlsx',index=False)

Генерируем наборы фраз 

In [8]:
import pandas as pd
data_42 = pd.read_excel('data_food_names.xlsx')
res_tags =[]
verb_num = 0
verbs = ['приготовить','попробовать','продегустировать', 'сфотографировать', 'научиться делать',
         'научиться готовить', 'съесть']
for i in data_42.iloc[:][0]:
#     print(i)
    res_tags.append([verbs[verb_num], i[0].lower() + i[1:]])
    verb_num += 1
    if verb_num > 6:
        verb_num = 0
print(len(res_tags))      
res_tags

17


[['приготовить', 'мясо в крахмале по-китайски'],
 ['попробовать', 'суп со шпинатом и соусом'],
 ['продегустировать', 'креветки в кисло-сладком соусе'],
 ['сфотографировать', 'битые огурцы'],
 ['научиться делать', 'гречка в азиатском стиле'],
 ['научиться готовить', 'гречневая лапша с курицей терияки'],
 ['съесть', 'рис с курицей и ананасом'],
 ['приготовить', 'салат из битых огурцов'],
 ['попробовать', 'стеклянная лапша с креветками'],
 ['продегустировать', 'китайские квадратики с фаршем'],
 ['сфотографировать', 'жареный рис по-китайски'],
 ['научиться делать', 'куриная грудка в кисло-сладком соусе'],
 ['научиться готовить', 'фунчоза с овощами и морепродуктами'],
 ['съесть', 'говядина по-сычуаньски'],
 ['приготовить', 'китайское печенье с предсказаниями'],
 ['попробовать', 'кисло-сладкие свиные ребра по-китайски'],
 ['продегустировать', 'свинина в кисло-сладком соусе с ананасом']]

In [9]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Функции ниже пытаются обработать исключения, которые очень мешались мне на пути. Довольно костыльным методом пришлось их решать, но это помогло 

In [10]:
def exclusion(p,p_next):
    if (p.word == 'панакота'):
        p = morph.parse(p.word)[2]
    elif (p_next.word == 'панакота'):
        p_next = morph.parse(p_next.word)[2]
    return p,p_next

             

def full_phrase_one(p,j, new_p):
    split = j[1].split(p)
    
    if (split[0]) == j[1]:
        p =p.replace('ё','е')
        split = j[1].split(p)
    try:
        normalaze_phrase = ' '.join([j[0],split[0],new_p, split[1]])
    except:
        normalaze_phrase = ' '.join([j[0],split[0],new_p])
    print(normalaze_phrase)
    
    return normalaze_phrase


def full_phrase_two(p,p_next,j, new_p,new_p_next):
    base = p+' '+p_next 
    split = j[1].split(base)

    if (split[0]) == j[1]:
        base = base.replace('ё','е')
        split = j[1].split(base)
    try:
        normalaze_phrase = ' '.join([j[0],split[0],new_p,new_p_next, split[1]])
    except:
        normalaze_phrase = ' '.join([j[0],split[0],new_p,new_p_next])
    print(normalaze_phrase)
    return normalaze_phrase




In [11]:
exclusion_dict = ['лазанья']


def exclusion_hard(j,p=None,p_next=None):
#     часть для одного слова 
    
#     print(f'p_next = {p_next}')
    if (p != None) and (p_next == None) or((p == None) and (p_next != None)):
        if (p == 'лазанья'):
            return full_phrase_one('лазанья', j,'лазанью')
        elif (p_next == 'лазанья'):
            return full_phrase_one('лазанья', j,'лазанью')
        
        elif (p == 'панакотта'):
            return full_phrase_one('панакотта', j,'панакотту')
        elif (p_next == 'панакотта'):
            return full_phrase_one('панакотта', j,'панакотту')
        
#     часть для двух слов 
    elif (p != None) and (p_next != None):
        
        if (p == 'лазанья'):
            new_p_next = morph.parse(p_next)[0].inflect({"femn","accs"}).word
            return full_phrase_two('лазанья',p_next,j,'лазанью',new_p_next)
        
        elif (p_next == 'лазанья'):
            new_p = morph.parse(p)[0].inflect({"femn","accs"}).word
            return full_phrase_two(p,'лазанья',j,new_p,'лазанью')
        
        if (p == 'панакотта'):
            new_p_next = morph.parse(p_next)[0].inflect({"femn","accs"}).word
            return full_phrase_two('панакотта',p_next,j,'панакотту',new_p_next)
        
        elif (p_next == 'панакотта'):
            new_p = morph.parse(p)[0].inflect({"femn","accs"}).word
            return full_phrase_two(p,'панакотта',j,new_p,'панакотту')

        


def replace_word(j):
    if 'панна  котта' in j[1]:
        return j[1].replace('панна  котта', 'панакота')
    elif 'панакотта' in j[1]:
        return j[1].replace('панакотта', 'панакота')
    elif 'панна-котта' in j[1]:
        return j[1].replace('панна-котта', 'панакота')
    else:
        return j[1]

In [12]:
exceptions_plur = ['панкейки','капкейки','чизкейки','маффины']
def exceptions(word):
#     print(word)
    if word.word in exceptions_plur:
        print('++++++')
        try:
            return word.inflect({"accs",'plur'}).word
        except:
            return word.word
    else:
        return word.word
    
    


Ниже мы описываем правилами все возможные варинаты склонения. 

In [13]:
count = 0
results = []

for j in res_tags[1:]:
    j[1] = replace_word(j)
    text = j[1].split()
    
#     print(text)
    
    for i in range(len(text))[:-1]:
        p = morph.parse(text[i])[0]
        p_next = morph.parse(text[i+1])[0]
        p,p_next = exclusion(p,p_next)
        if ('ADJF' in p.tag) and ('NOUN' in p_next.tag) and ('sing' in p_next.tag) and ('nomn' in p_next.tag):
            if (p_next.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p.word,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
            
            
            elif ('masc' in p.tag) or ('neut' in p.tag):
                normalaze_phrase = full_phrase_two(p.word,p_next.word,j, p.word,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            else:
                normalaze_phrase = full_phrase_two(p.word,p_next.word,j,p.inflect({'sing',"accs"}).word,p_next.inflect({'sing',"accs"}).word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            break

#  сущ + прилаг (наоборот от первого if)    
        if ('ADJF' in p_next.tag) and ('NOUN' in p.tag) and ('sing' in p.tag) and ('nomn' in p.tag):
            if (p.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p.word,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            elif ('masc' in p.tag) or ('neut' in p.tag):
                normalaze_phrase = full_phrase_two(p.word,p_next.word,j, p.word,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            else:
                normalaze_phrase = full_phrase_two(p.word,p_next.word,j, p.inflect({'sing',"accs"}).word,p_next.inflect({'sing',"accs"}).word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            break
    
            
        elif ('ADJF' in p.tag) and ('NOUN' in p_next.tag) and ('plur' in p_next.tag) and ('nomn' in p_next.tag):
            if (p_next.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p.word,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
            
            else:
                normalaze_phrase = full_phrase_two(p.word,p_next.word,j, p.word,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
    
            break
                               
#           отдельно проверям целую строку и отдельно последний символ.
        elif ('NOUN' in p.tag) and ('sing' in p.tag): #одно сущ  (без последнего слова)
            
            if (p.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p=p.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            elif (p.word in exceptions_plur):
                normalaze_phrase = full_phrase_one(p.word,j,exceptions(p))
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                
                
            elif ('femn' in p.tag):#женского рода
                normalaze_phrase = full_phrase_one(p.word,j,p.inflect({'sing',"accs"}).word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            else:
                normalaze_phrase = full_phrase_one(p.word,j,p.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")

            break
            
           
        elif ('NOUN' in p_next.tag) and ('sing' in p_next.tag) :
            if (p_next.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p_next=p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            elif (p_next.word in exceptions_plur):
                normalaze_phrase = full_phrase_one(p_next.word,j,exceptions(p_next))
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                
            elif ('femn' in p_next.tag):#женского рода
                normalaze_phrase = full_phrase_one(p_next.word,j,p_next.inflect({'sing',"accs"}).word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            else:
                normalaze_phrase = full_phrase_one(p_next.word,j,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            break
        
        
        elif ('NOUN' in p.tag) and ('plur' in p.tag) and ('nomn' in p.tag): #одно сущ  м.ч (без последнего слова)
            if (p.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p=p.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            else:
                normalaze_phrase = full_phrase_one(p.word,j,p.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")                            
                                      
            break
            
           
        elif ('NOUN' in p_next.tag) and ('plur' in p_next.tag) and ('nomn' in p_next.tag):
            if (p_next.word in exclusion_dict):
                normalaze_phrase = exclusion_hard(j,p_next=p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")
                               
            else:
                normalaze_phrase = full_phrase_one(p_next.word,j,p_next.word)
                results.append(f"{'[' + ', '.join(j) + ']'} => {normalaze_phrase.replace('  ',' ')}")

        break
            

попробовать  суп  со шпинатом и соусом
продегустировать  креветки  в кисло-сладком соусе
сфотографировать  битые огурцы 
научиться делать  гречку  в азиатском стиле
научиться готовить  гречневую лапшу  с курицей терияки
съесть  рис  с курицей и ананасом
приготовить  салат  из битых огурцов
попробовать  стеклянную лапшу  с креветками
продегустировать  китайские квадратики  с фаршем
сфотографировать жареный  рис  по-китайски
научиться делать  куриную грудку  в кисло-сладком соусе
научиться готовить  фунчоза  с овощами и морепродуктами
съесть  говядину  по-сычуаньски
приготовить  китайское печенье  с предсказаниями
продегустировать  свинину  в кисло-сладком соусе с ананасом


In [14]:
# 1. прилаг + сущ (ед, ж.р.) -> оба меняются в В.п. 
# 2. прилаг + сущ (ед, м.р. И вроде с.р.) ->  не меняется 
# 3. прилаг + сущ (мн) - > не меняется 
# 4. сущ (ед) -> меняется сущ в В.п.
# 5. сущ (мн)- > не меняется 
# 6. сущ (название) -> вроде не меняется 

In [15]:

input_tags = []
output = []
for i in results:
    input_tags.append(i.split('=>')[0])
    output.append(i.split('=>')[1])

data_in = pd.Series(input_tags)
data_out = pd.Series(output)

data_res = pd.DataFrame(zip(data_in,data_out))

data_res = data_res.rename({0: 'inputs', 1: 'outputs'}, axis=1)


data_res.to_excel('grammar_outputs_food.xlsx', sheet_name = 'data', index=False)